# Project Title <img src="https://media.licdn.com/dms/image/C4E0BAQFJk4Ibe9N3lg/company-logo_400_400/0?e=1576108800&v=beta&t=aWqzP3WNVsDqWJVyqZrWMY86jlCjtnVZ1-AbKzUFzfI" width=26 height=26 align="right"> </a>



Case de Flávio de Assis para a vaga de Engenheiro de Dados. Esse notebook responderá aos questionamentos definidos pelo case.

### Fonte de Dados
<ul>
    <li> gazeus_data.csv - CSV fornecido pela empresa com dados reais de utilização de clientes

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

### Localização do arquivo

In [2]:
file_url = 'files/gazeus_data.csv'
df = pd.read_csv(file_url, low_memory=False)

### Overview dos Dados

In [3]:
df.head()

,app_name,created_at_date,installed_at_date,event_name,event_code,country,user_id
0,BURACO_ANDROID,2019-09-01 00:43:57,2019-08-30 19:03:36,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,0
1,BURACO_ANDROID,2019-09-03 01:55:02,2019-09-03 01:15:52,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,1
2,BURACO_ANDROID,2019-09-01 00:32:05,2019-08-31 14:01:49,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,2
3,BURACO_ANDROID,2019-09-01 00:35:06,2019-08-30 02:00:45,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,3
4,BURACO_ANDROID,2019-09-01 00:04:40,2019-08-27 21:22:26,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,4


In [4]:
df.dtypes

app_name             object
created_at_date      object
installed_at_date    object
event_name           object
event_code           object
country              object
user_id              object
dtype: object

In [5]:
df.shape

(1000000, 7)

### Perguntas:

O case deve ter 6 perguntas respondidas, são elas:

<ol>
    <li> Quantos usuários estão presentes nesta massa? Quantos eventos? (responda com os números e qualquer código que tenha gerado para chegar a eles)</li>

<li>Qual plataforma tem mais sessões de jogo? Considere que a plataforma faz parte do app_name, podendo ser “ANDROID” ou “IPHONE”</li>

<li>Quantos usuários que assistiram o tutorial abandonaram o jogo e quantos permaneceram? Considere abandono aquele usuário que não teve mais eventos após jogar por 3 dias ou mais.
LEGENDA: "tutorial_start" significa começou o tutorial tutorial_end" significa terminou tutorial com sucesso, "tutorial_quit" é abandonou o tutorial
OBS: queremos abandono DO JOGO, não do tutorial</li>

<li>Quais são os usuário que jogaram até 5 dias e pararam? Providencie o código em Python que retorne uma lista com os user_id apropriados e quanto tempo cada um jogou em horas.</li>

<li>Qual a média e desvio padrão de tempo jogado (em horas) do subconjunto que você gerou acima?</li>

<li>Considerando que uma partida é o intervalo entre um evento de match_start e match_end de um mesmo usuário e levando em conta ainda que a quantidade de match_start e match_end podem não ser iguais (você pode definir o tratamento que achar mais conveniente para chegar a um resultado, descreva-o na solução): Qual é o tempo médio entre as partidas e o desvio padrão?</li>


### Estratégia de Resolução

Para a pergunta 1, deve-se contabillizar o número único de user_id presentes no dataset. Para responder a pergunta de quantos eventos, basta contabilizar o número total de linhas válidas na tabela, uma vez que cada uma represeta um evento.

### Limpeza do dado:

Para verificar a qualidade das variáveis categóricas, verifica-se os valores únicos presentes no DataFrame para estas colunas

In [6]:
for i in df['app_name'].unique():
    print(i)

BURACO_ANDROID
SPADES_IPHONE
BURACO_IPHONE
SPADES_ANDROID
BURACO-ANDROID


In [7]:
for i in df['event_name'].unique():
    print(i)

impression_standard
match_end
match_start
match_quit
tutorial_start
tutorial_quit
tutorial_end
match(_end
match_)end


In [8]:
for i in df['event_code'].unique():
    print(i)

0db601a32123556ffaec586e32fb51f7c293124b
8e96f5ed27fb1511a7a0e4785eef276a8a543f36
0815ffebc19b4bfca409284fff97376b574b67ce
78278a4d1aa7ab1300e24e0a2f75ffc1bc21e604
86c914c92e664c4b9f26dbc98eff283492967ccd
ab7faf2810718e30a095c6fe36b7a267367625bc
4eff55d3d6e1636a75548de1e387439fdbcfb1c5
b471abe61ebcd5b468c180e4947166df86125cf7
0799dd02335be65d8a71f5f3d6d473456dcb3c2c
bdb223890d3a9b5d65563c9d71e977eebbc27e06
9b9a5014c642d827d3d11d07ceb0936f950c377d
76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7
f8343507810b98d36d907f5c70609b86392ff062
efe9d3a0cf169d7e3c60e7150a59aa8b49a1f3e1


In [9]:
for i in df['country'].unique():
    print(i)

br
us
ar
pt
tr
pe
ae
ir
vi
au
es
gr
ca
ge
de
qa
py
vn
co
jp
uy
cl
pr
cy
it
in
mm
mx
gb
sa
bo
ve
th
cn
ht
lb
do
iq
ie
rs
fr
nl
bd
az
ao
sy
nz
se
om
ee
ch
bg
ma
eg


Aparentemente existem mais de um código de evento para cada nome.
Por que?
<br>De qualquer forma, existem erros de sintaxe em <b>app_name</b> e em <b>event_name</b> que devem ser corrigido para que as futuras análises não sejam impactadas.

In [10]:
df.loc[(df['app_name']) == 'BURACO-ANDROID', 'app_name'] = 'BURACO_ANDROID'

In [11]:
for i in df['app_name'].unique():
    print(i)

BURACO_ANDROID
SPADES_IPHONE
BURACO_IPHONE
SPADES_ANDROID


In [12]:
df.loc[(df['event_name']).isin(['match_)end','match(_end']), 'event_name'] = 'match_end'

In [13]:
for i in df['event_name'].unique():
    print(i)

impression_standard
match_end
match_start
match_quit
tutorial_start
tutorial_quit
tutorial_end


In [14]:
df[[ 'app_name', 'event_name', 'event_code']].groupby(by=['event_name', 'event_code']).count()

app_name
event_name          event_code                                        
impression_standard 0db601a32123556ffaec586e32fb51f7c293124b    789457
                    8e96f5ed27fb1511a7a0e4785eef276a8a543f36     60099
match_end           0815ffebc19b4bfca409284fff97376b574b67ce     76611
                    b471abe61ebcd5b468c180e4947166df86125cf7      2044
match_quit          86c914c92e664c4b9f26dbc98eff283492967ccd      1158
                    ab7faf2810718e30a095c6fe36b7a267367625bc      7513
match_start         0799dd02335be65d8a71f5f3d6d473456dcb3c2c      3796
                    78278a4d1aa7ab1300e24e0a2f75ffc1bc21e604     48898
tutorial_end        9b9a5014c642d827d3d11d07ceb0936f950c377d      2423
                    efe9d3a0cf169d7e3c60e7150a59aa8b49a1f3e1       288
tutorial_quit       76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7      2242
                    bdb223890d3a9b5d65563c9d71e977eebbc27e06       278
tutorial_start      4eff55d3d6e1636a75548de1e387439fdbcfb1c5      4615
                    f8343507810b98d36d907f5c70609b86392ff062       578

Aparentemente há então dois códigos para cada evento. 
Por que? Seriam estes códigos únicos dependendo da plataforma do APP?

<p> Deve-se construir uma coluna com a plataforma do APP, para que dessa forma as análises futuras sejam possíveis.

In [15]:
df['platform'] = df.copy()['app_name'].map(lambda row: row.split('_')[1])

In [16]:
df.head()

,app_name,created_at_date,installed_at_date,event_name,event_code,country,user_id,platform
0,BURACO_ANDROID,2019-09-01 00:43:57,2019-08-30 19:03:36,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,0,ANDROID
1,BURACO_ANDROID,2019-09-03 01:55:02,2019-09-03 01:15:52,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,1,ANDROID
2,BURACO_ANDROID,2019-09-01 00:32:05,2019-08-31 14:01:49,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,2,ANDROID
3,BURACO_ANDROID,2019-09-01 00:35:06,2019-08-30 02:00:45,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,3,ANDROID
4,BURACO_ANDROID,2019-09-01 00:04:40,2019-08-27 21:22:26,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,4,ANDROID


In [17]:
df[[ 'app_name', 'event_name', 'event_code', 'platform']].groupby(by=['event_name', 'event_code', 'platform']).count()

app_name
event_name          event_code                               platform          
impression_standard 0db601a32123556ffaec586e32fb51f7c293124b ANDROID     652240
                                                             IPHONE      137217
                    8e96f5ed27fb1511a7a0e4785eef276a8a543f36 ANDROID      38747
                                                             IPHONE       21352
match_end           0815ffebc19b4bfca409284fff97376b574b67ce ANDROID      64746
                                                             IPHONE       11865
                    b471abe61ebcd5b468c180e4947166df86125cf7 ANDROID       1409
                                                             IPHONE         635
match_quit          86c914c92e664c4b9f26dbc98eff283492967ccd ANDROID        813
                                                             IPHONE         345
                    ab7faf2810718e30a095c6fe36b7a267367625bc ANDROID       6695
                                                             IPHONE         818
match_start         0799dd02335be65d8a71f5f3d6d473456dcb3c2c ANDROID       2581
                                                             IPHONE        1215
                    78278a4d1aa7ab1300e24e0a2f75ffc1bc21e604 ANDROID      41559
                                                             IPHONE        7339
tutorial_end        9b9a5014c642d827d3d11d07ceb0936f950c377d ANDROID       2114
                                                             IPHONE         309
                    efe9d3a0cf169d7e3c60e7150a59aa8b49a1f3e1 ANDROID        206
                                                             IPHONE          82
tutorial_quit       76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7 ANDROID       1966
                                                             IPHONE         276
                    bdb223890d3a9b5d65563c9d71e977eebbc27e06 ANDROID        199
                                                             IPHONE          79
tutorial_start      4eff55d3d6e1636a75548de1e387439fdbcfb1c5 ANDROID       4065
                                                             IPHONE         550
                    f8343507810b98d36d907f5c70609b86392ff062 ANDROID        418
                                                             IPHONE         160

Como esperado, cada um evento tem dois identificadores, um para cada plataforma que o app está instalado.

Há eventos repetidos no dataset, ou as linhas são únicas?


In [18]:
df.loc[df.duplicated(subset=None, keep='first')]

,app_name,created_at_date,installed_at_date,event_name,event_code,country,user_id,platform
13271,BURACO_ANDROID,2019-09-01 02:20:53,2019-09-01 02:01:09,tutorial_quit,76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7,br,875,ANDROID
23644,BURACO_IPHONE,2019-09-03 00:29:57,2019-09-03 00:27:17,tutorial_quit,76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7,br,4754,IPHONE
34214,BURACO_ANDROID,2019-09-01 02:20:45,2019-09-01 02:01:09,tutorial_quit,76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7,br,875,ANDROID
55112,BURACO_ANDROID,2019-08-31 20:55:57,2019-08-26 13:00:31,match_start,78278a4d1aa7ab1300e24e0a2f75ffc1bc21e604,br,1793,ANDROID
56679,BURACO_ANDROID,2019-08-31 20:55:57,2019-08-26 13:00:31,match_start,78278a4d1aa7ab1300e24e0a2f75ffc1bc21e604,br,1793,ANDROID
...,...,...,...,...,...,...,...,...
917134,BURACO_ANDROID,2019-09-03 00:19:33,2019-09-03 00:12:42,tutorial_quit,76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7,br,18814,ANDROID
925344,SPADES_ANDROID,2019-09-03 00:31:05,2019-08-25 03:51:08,impression_standard,8e96f5ed27fb1511a7a0e4785eef276a8a543f36,us,24521,ANDROID
933025,BURACO_ANDROID,2019-09-02 22:13:56,2019-09-01 23:23:00,impression_standard,0db601a32123556ffaec586e32fb51f7c293124b,br,21986,ANDROID
989640,BURACO_ANDROID,2019-08-27 03:05:04,2019-08-26 03:20:03,tutorial_quit,76b5ede3ae7edbef4fb55521eb2f9662fd5a95c7,br,13632,ANDROID


In [19]:
df.drop_duplicates(inplace=True)
df.shape

(999893, 8)

Com visto no resultado acima, há <b>107</b> eventos duplicados. Pode-se concluir então que há <b>999893</b> eventos no dataset.

Deve-se retirar estes eventos:

Verificando o número de usuários únicos.

In [20]:
len(df['user_id'].unique())

25081

Portanto, respondendo:

1) Há <b>25081</b> usuários presentes na massa e <b>999893</b> eventos.

Uma das formas de responder a pergunta 2) é verificar o número de sessões agrupadas pela plataforma.
Considerarei, que as sessões de jogos podem ser definidas pelo número de eventos que a plataforma têm, pois no grande escopo os eventos tendem a mostrar a maior utilização de uma determinada plataforma.

In [21]:
df.groupby(by='platform').count()

,app_name,created_at_date,installed_at_date,event_name,event_code,country,user_id
platform,,,,,,,
ANDROID,817655,817655,817655,817655,817655,817655,817655
IPHONE,182238,182238,182238,182238,182238,182238,182238


Desta forma, a plataforma <b>ANDROID</b> é a que possui mais sessão de jogos com 817655 eventos contra 182238 do <b>IPHONE</b>

Para responder 3) primeiro transforma-se a coluna created_at_date em formato de datetime do python para que as análises sejam simplificadas.

In [22]:
df['created_at_date'] = pd.to_datetime(df['created_at_date'])

In [23]:
df.dtypes

app_name                     object
created_at_date      datetime64[ns]
installed_at_date            object
event_name                   object
event_code                   object
country                      object
user_id                      object
platform                     object
dtype: object

Após a modificação feita, filtra-se para ver quais os usuários terminaram o tutorial

In [25]:
tutorial_finished = df.copy().loc[df.copy()['event_name'] == 'tutorial_quit']

In [26]:
tutorial_finished.shape

(2496, 8)

In [27]:
len(tutorial_finished['user_id'].unique())

2306

Ou seja, <b>2306</b> usuários únicos assistiram o tutorial.

Agora, define-se um dataframe com as últimas mofificações por <b>user_id</b>

In [28]:
last_events = df.sort_values('created_at_date', ascending=False).drop_duplicates(subset=['user_id'])
last_events.shape

(25081, 8)

In [29]:
tutorial_finished = tutorial_finished.sort_values('created_at_date', ascending=False).drop_duplicates(subset='user_id')
tutorial_finished.shape

(2306, 8)

Fazendo um merge entre last_events e tutorial_finished para verificar qual foi o ultimo evento dos usuários que terminaram o tutorial:

In [30]:
comparison_df = tutorial_finished.merge(last_events[['user_id', 'created_at_date', 'event_name']], on='user_id', suffixes=('','_last'))

Excluindo aqueles que tiveram como último evento o termino do turial

In [31]:
comparison_df = comparison_df.loc[comparison_df['event_name_last']!=comparison_df['event_name']]
comparison_df.shape

(2149, 10)

In [32]:
def abandon_game(line):
    return line['created_at_date_last']-line['created_at_date'] < timedelta(3)

Crio agora uma coluna com valor booleano para identificar se o evento ocorreu <b> 3 dias ou menos</b> o que indica que o usuário não jogou depois do prazo de 3 dias para consideração de abandono. 

In [33]:
comparison_df['abandon'] = comparison_df.apply(lambda row: abandon_game(row), axis=1)

In [34]:
comparison_df[comparison_df['abandon']==True].shape

(1974, 11)

Ou seja, <b>1974</b> usuários abandonaram o jogo após assistir o tutorial. (considerando que seu ultimo evento ocorreu em até 3 dias depois de o tê-lo assistido)

Definindo a função pedida em 4) :

In [39]:
def time_hours(user_id):
    print(user_id)
    initial = df.loc[df['user_id']==user_id].sort_values('created_at_date')['created_at_date'].iloc[0]
    end = df.loc[df['user_id']==user_id].sort_values('created_at_date', ascending=False)['created_at_date'].iloc[0]
    return [user_id, round((end-initial).seconds/3600,2)]


Descobrindo o número de pessoas que jogaram até 5 dias e depois pararam:

In [ ]:
series = [time_hours(user) for user in df['user_id'].unique()]

In [ ]:
user_hours = pd.DataFrame

In [ ]:
more_5days = user_hours[user_hours['hours'] > 120]
print(more_5days['hours'].mean())
print(more_5days['hours'].std())

### Concluões